In [28]:
# Let's read the data_file into memory
spect_data = data.frame(read.csv("/home/srinivas/Git-Projects/spect-team/Data/Data_Adults_1.csv"))

# Drop NAs
spect_data = na.omit(spect_data)

# Extract the baseline and Concentration Values
interesting_baseline_column_start = which(colnames(spect_data) == "Baseline_Amygdala_L")
interesting_baseline_column_end = which(colnames(spect_data) == "Baseline_Vermis_9")
interesting_concentration_column_start = which(colnames(spect_data) == "Concentration_Amygdala_L")
interesting_concentration_column_end = which(colnames(spect_data) == "Concentration_Vermis_9")
baseline_values = spect_data[interesting_baseline_column_start : interesting_baseline_column_end]
concentration_values = spect_data[interesting_concentration_column_start : interesting_concentration_column_end]

# Prefix the Patient Id to make them uniquely recongizable
b_vals = cbind(spect_data["Patient_ID"], baseline_values)
c_vals = cbind(spect_data["Patient_ID"], concentration_values)

# Let us also compute the deltas in the blood flow
delta_values = concentration_values - baseline_values
deltas = cbind(spect_data["Patient_ID"], delta_values)

#deltas[1:4, 1:4]

Now that we have the the RCBF value let us read in 2 lists, a list of healthy patients and a list of people who have brain trauma

** EDIT THE FILE PATHS TO POINT TO A FILE ON YOUR PC **

In [43]:
healthy_patients = data.frame(read.csv("/home/srinivas/Git-Projects/spect-team/Data/healthy_IDs.csv"))
anxious_patients = data.frame(read.csv("/home/srinivas/Git-Projects/spect-team/Data/AnxietyDisorder_IDs.csv"))


We have 50 healthy people and 15 people with Anxiety Disorder, let us now get the RCBF values for these individuals

In [46]:
# Let us perform a cartesian join of the two data frames
healthy = deltas[deltas$Patient_ID %in% healthy_patients$Patient_ID,]
anxious = deltas[deltas$Patient_ID %in% anxious_patients$Patient_ID,]

Now we don't know which rcbf delta corresponds is an indicator that a person suffers from a certain disorder,
We propose running a series of Wilcoxon-Mann-Whitney rank sum test on all the columns

In [74]:
p_vector = 0
# Lets loop over the columns
for (i in names(healthy[-1])) {
    col1 = as.vector(healthy[,i])
    col2 = as.vector(anxious[,i])
    ans = wilcox.test(col1, col2, alternative="two.sided")
    p_vector = cbind(p_vector, ans[3])
}
names(p_vector) = names(healthy)
write.csv(healthy, "healthy.csv")
write.csv(anxious, "anxious.csv")
write.csv(p_vector, "pvals.csv")

        p_vector                                                            
p.value 0        0.4408765 0.5911541 0.6348694 0.8456867 0.6572274 0.8092957
                                                                             
p.value 0.5911541 0.9565409 0.5181795 0.7614215 0.8092957 0.5911541 0.7972524
                                                                              
p.value 0.7852571 0.5283125 0.7733126 0.02758404 0.3790198 0.1498043 0.4982194
                                                                            
p.value 0.1140261 0.8947116 0.9070385 0.944142 0.5283125 0.5385454 0.9813671
                                                                              
p.value 0.6348694 0.9689502 0.9565409 0.08821071 0.7144438 0.9565409 0.5804454
                                                                             
p.value 0.9070385 0.9689502 0.9689502 0.9317567 0.7028586 0.9317567 0.6128396
                                                                